In [190]:
import json
import jieba
import pandas as pd
import pyprind
import re
import multiprocessing

class WordCut():
    # process sentence and cut if with jieba
    def cut_word(self,n):
        inifile = open('../data/split_file00{}'.format(n), 'r')
        csvfile = open('../data/csvfile00{}'.format(n), 'w')
        writer = csv.writer(csvfile)
        for l in inifile:
            content = json.loads(l)
            score = content['comment_detail']['score']
            comment = content['comment_detail']['content']
            comment = re.sub('[\W]+', ' ', comment)
            # 分词之后的调整
            if not (score and comment.strip()): # 剔除掉空值。错了！ 应该为and，任何一个为空则跳过
                continue
            if comment[:3] == '此用户':
                continue
            sentence = jieba.cut(comment)
            sentence = ' '.join(sentence)
            label = 0 if score <= 3 else 1 # 小于等于3为反例（负面）
            writer.writerow([sentence, label])
        inifile.close()
        csvfile.close()
        
    def run(self):
        ps = list()
        for i in range(6):
            p = multiprocessing.Process(target=self.cut_word, args=(i,), name=i)
            ps.append(p)
            p.start()
            print("process {} start".format(i))
        for p in ps:
            p.join()
            print("process {} joined".format(p.name))

In [191]:
WC = WordCut()
WC.run()

process 0 start
process 1 start
process 2 start
process 3 start
process 4 start
process 5 start
process Process-105 joined
process 1 joined
process 2 joined
process 3 joined
process 4 joined
process 5 joined


In [192]:
# save data to local csv file
import numpy as np
np.random.seed(0)
df_all = pd.DataFrame(columns=['sentence', 'label']) # append不同dataFrame时，column应保持一致
for i in range(6):
    df = pd.read_csv('../data/csvfile00{}'.format(i), delimiter=',')
    df.columns = ['sentence', 'label']
    print(df.shape)
    df_all = df_all.append(df, ignore_index=True)
# reindex data
df_all = df_all.reindex(np.random.permutation(df_all.index))
df_all.to_csv('../data/csvfinal.csv', index=False)

(76487, 2)
(74097, 2)
(75196, 2)
(76778, 2)
(78202, 2)
(40803, 2)


In [193]:
df = pd.read_csv('../data/csvfinal.csv')
# df.drop_duplicates() # 删除重复数据
# df.isnull().sum() # 判断空值
# df.dropna()
# df.sentence.astype==float
df['sentence'].head()

0                                    不能 关 数据   感觉 好 垃圾
1                                    手机 便宜   但 声音 有点 小
2    用 了 三天   非常 好   还是 htc 不 一样 的 感觉   非常 流畅   昨天 ...
3    手机 给 老妈 用 的   使用 后 反馈 还 行   声音 比较 大   字体 清楚   ...
4    买 了 一个 星期 多 了   游戏 也 玩 了   没 出现 卡机 现象   屏幕 有 水...
Name: sentence, dtype: object

In [194]:
# load stop_words
stopwords = open('../data/stop_words.txt', 'r', encoding='GBK').read() #打不开则用GBK编码， 默认使用utf8
stops = stopwords.splitlines()

In [195]:
# transform sentence to list/array
count = 0
def tokenizer(text):
    try:
        return text.split() # 以任意空格分割句子
    except Exception as e:
        print(e)
        print(text)
        global count
        count += 1
        return []

In [196]:
# 将数据分为测试集与训练集
from sklearn.cross_validation import train_test_split
X = df.iloc[:, 0].values
y = df.iloc[:,1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [197]:
X_train[y_train==0]

array(['之前 的 坏 了   买 了 一个 先用 用   屏幕 看上去 不像 正品 的   耗电量 巨快', '服务 差劲',
       '非常 不 满意 的 一次 网购   买 之前 看 评论   普遍 都 觉得 还 不错   而且 是 给 老人 买   我 觉得 这些 配置 应该 可以 了   可 手机 拿到 半天 不到   按键 迟钝   滑动 缓慢   应用 点不开   客服 给 我 的 官方 解释 是   手机 需要 垃圾 清理   我 想 请问   新 到手 的 机子 就 开始 清 垃圾   这能 用 几天   这 还要 在 我 能点 进去 的 情况 下  ',
       ...,
       '手机 有 问题   不仅 不 退款   而且 也 不 给 退换   找 了 个 手机 有 划痕 的 借口   退返 厂家 检测 快递费 还是 自付 的   最后 还是 退 给 我 之前 有 问题 的 手机   希望 大家 别 上当   换家 靠 谱 的 购物 消费 吧',
       '飞远 快递 配送 的   不能 送上门   叫 老人家 跑 10 公里 远 地方 去 拿   投诉 后 还 打电话 来 骂',
       '卡成 狗 了   玩 游戏卡 到 爆   你妹 的   16G 内存   竟然 打 不了 王者   等 放出 技能 人 都 死 啦  '],
      dtype=object)

In [199]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
clf = LogisticRegression(random_state=0)
tfidf = TfidfVectorizer(preprocessor=None, lowercase=None, strip_accents=None, stop_words=stops, tokenizer=tokenizer)
lr_tfidf = Pipeline([('vect', tfidf), ('clf', clf)])
params_grid = [{'vect__ngram_range': [(1,1)],
      'vect__norm': ['l1', 'l2'], # 数据归一化
      'clf__penalty': ['l1', 'l2'], # logistic回归罚项
      'clf__C': [1.0, 10.0, 100.0]
     },                  
    {'vect__ngram_range': [(1,1)],
     'vect__use_idf': [False], # 不使用idf
     'vect__norm': [None],
     'clf__penalty': ['l1', 'l2'],
     'clf__C': [1.0, 10.0, 100.0]}]
gs_lr_tfidf = GridSearchCV(lr_tfidf, params_grid, scoring='accuracy', cv=5, verbose=-1, n_jobs=-1)
gs_lr_tfidf.fit(X_train, y_train)
# matrix = tfidf.fit_transform(X_test)
# vocdict = tfidf.vocabulary_
# invers_dict = {v: k for k, v in vocdict.items()}
# print(matrix)
# print(invers_dict[11745])


[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed: 149.9min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=None, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...nalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid=[{'vect__ngram_range': [(1, 1)], 'vect__norm': ['l1', 'l2'], 'clf__penalty': ['l1', 'l2'], 'clf__C': [1.0, 10.0, 100.0]}, {'vect__ngram_range': [(1, 1)], 'vect__use_idf': [False], 'vect__norm': [None], 'clf__penalty': ['l1', 'l2'], 'clf__C': [1.0, 10.0, 100.0]}],
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=-1)

In [205]:
gs_lr_tfidf.best_score_

0.9586504639199712

In [202]:
clf = gs_lr_tfidf.best_estimator_

In [203]:
clf

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=None, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...nalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [208]:
clf.score(X_test, y_test)

0.9583138951047292

In [216]:
clf.predict(['手机 开机 很 慢，反应 很 慢'])

array([1])

In [217]:
print(y_train.count(0))

AttributeError: 'numpy.ndarray' object has no attribute 'count'